# **Building Random Forest Classifier from Scratch**

In [ ]:
import random
import pandas as pd
import numpy as np

df = pd.read_csv("/content/titanic.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


The datast consists of simple attributes for each passenger like their age, sex, social class, # of family members, and where they embarked. The objective is to predict whether a passenger survived the titanic crash or not, where 1 denotes that the passenger survived and 0 denotes that they perished.

In [ ]:
## handling missing values
df.loc[df['Age'].isnull(),'Age'] = np.round(df['Age'].mean())
df.loc[df['Embarked'].isnull(), 'Embarked'] = df['Embarked'].value_counts().index[0]

In [ ]:
df.drop('Cabin', axis=1, inplace=True)

In this implementation I'll use 7 features: Pclass, Sex, Age, SibSp, Parch, Fare, Embarked

In [ ]:
df.drop(['Name', 'Ticket','PassengerId'], axis=1, inplace=True)

In [ ]:
## Splitting the dataset
from sklearn.model_selection import train_test_split

X = df.drop('Survived',axis=1)
y = df['Survived']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=217)

Let's first define **Entropy** and **Information Gain** which will help us in finding the best split point

In [ ]:
def entropy(p):
  if p==0:
    return 0
  elif p==1:
    return 0
  else:
    return - (p * np.log2(p) + (1-p) * np.log2(1-p))

def information_gain(left_child, right_child):
  parent = left_child + right_child

  p_parent = parent.count(1)/len(parent) if len(parent)>0 else 0
  p_left = left_child.count(1)/len(left_child) if len(left_child)>0 else 0
  p_right = right_child.count(1)/len(right_child) if len(right_child)>0 else 0

  entropy_parent = entropy(p_parent)
  entropy_left = entropy(p_left)
  entropy_right = entropy(p_right)

  IG = entropy_parent - len(left_child)/len(parent)*entropy_left - len(right_child)/len(parent)*entropy_right

  return IG

Lets also define a **draw_bootstrap** function that can take in the training input X in the form of a dataframe and also the output y in the form of an array. We'll have it return the bootstrap sampled Xboot and yboot that we'll use to construct a tree.

We'll also return the **out-of-bag** observations that were left out for training which we'll call **X_oob** and **y_oob**. At each new iteration we'll use the OOB samples to evaluate the performance of the tree built with the bootstrapped data.

In [ ]:
def draw_bootstrap(X_train, y_train):
  bootstrap_indices = list(np.random.choice(range(len(X_train)), len(X_train), replace=True))
  oob_indices = [i for i in range (len(X_train)) if i not in bootstrap_indices]

  X_bootstrap = X_train.iloc[bootstrap_indices].values
  y_bootstrap = y_train.iloc[bootstrap_indices]
  X_oob = X_train.iloc[oob_indices].values
  y_oob = y_train.iloc[oob_indices]

  return X_bootstrap, y_bootstrap, X_oob, y_oob

def oob_score(tree, X_test, y_test):
  mis_label = 0
  for i in range (len(X_test)):
    pred = predict_tree(tree, X_test[i])
    if pred != y_test[i]:
      mis_label += 1

  return mis_label/len(X_test)


Next we'll define find_split_point which does the following:

1] select m features at random

2] for each feature selected, iterate through each value in the bootstrapped dataset and compute the information gain

3] Return a dictionary from the value that gives the highest information gain, which will represent a node in our tree consisting of:

(int) the feature index

(int) the value to split at

(dictionary) left child node

(dictionary) right child node

In [ ]:
def find_split_point(X_bootstrap, y_bootstrap, max_features):
  feature_lst = []
  num_features = len(X_bootstrap[0])
  X_bootstrap = np.array(X_bootstrap)

  while len(feature_lst) != max_features:
    feature_index = random.sample(range(num_features),1)
    if feature_index not in feature_lst:
      feature_lst.extend(feature_index)

  best_info_gain = -999
  node = None

  for feature_idx in feature_lst:
    for split_point in X_bootstrap[:, feature_idx]:
      left_child = {"X_bootstrap": [], "y_bootstrap": []}
      right_child = {"X_bootstrap": [], "y_bootstrap": []}

      # spliting child node for continuous variable
      if type(split_point) in [int, float]:
        for i, value in enumerate(X_bootstrap[:, feature_idx]):
          if value <= split_point:
            left_child['X_bootstrap'].append(X_bootstrap[i])
            left_child['y_bootstrap'].append(y_bootstrap[i])
          else:
            right_child['X_bootstrap'].append(X_bootstrap[i])
            right_child['y_bootstrap'].append(y_bootstrap[i])

      # splitting child node for categorical variable
      else:
        for i, value in enumerate(X_bootstrap[:, feature_idx]):
          if value == split_point:
            left_child['X_bootstrap'].append(X_bootstrap[i])
            left_child['y_bootstrap'].append(y_bootstrap[i])
          else:
            right_child['X_bootstrap'].append(X_bootstrap[i])
            right_child['y_bootstrap'].append(y_bootstrap[i])

      split_IG = information_gain(left_child['y_bootstrap'], right_child['y_bootstrap'])

      if split_IG > best_info_gain:
        best_info_gain = split_IG
        left_child['X_bootstrap'] = np.array(left_child['X_bootstrap'])
        right_child['y_bootstrap'] = np.array(right_child['y_bootstrap'])
        node = {'information_gain': split_IG,
                'left_child': left_child,
                'right_child': right_child,
                'split_point': split_point,
                'feature_index': feature_idx}

    return node

a terminal node (classifies whether the passenger survives or perishes).

In [ ]:
def terminal_node(node):
  y_bootstrap = list(node['y_bootstrap'])
  prediction = max(y_bootstrap, key = y_bootstrap.count)
  return prediction

We'll next need a function that decides when to stop splitting nodes in a tree and finally output a terminal node. On a single tree split_node works as follows:

In [ ]:
def split_node(node,  max_features, min_sample_split, max_depth, depth):
  left_child = node['left_child']
  right_child = node['right_child']

  del(node['left_child'])
  del(node['right_child'])

  # 1] Check if either children has 0 observations in them. If one of the children is entirely empty this ultimately means that the best split in the data for that node was unable to differentiate the 2 classes and its best to call terminal_node and return the tree. terminal_node returns the class with the highest counts at the current node.
  if len(left_child['y_bootstrap'])==0 or len(right_child['y_bootstrap'])==0:
    if len(left_child['y_bootstrap'])!=0:
      empty_node = {'y_bootstrap': left_child['y_bootstrap']}
    else:
      empty_node = {'y_bootstrap': right_child['y_bootstrap']}

    node['left_split'] = terminal_node(empty_node)
    node['right_split'] = terminal_node(empty_node)
    return node

  # 2] Check if the current depth of the tree has reached the maximum depth. If so, create a terminal node and return the tree
  if depth >= max_depth:
    node['left_split'] = terminal_node(left_child)
    node['right_split'] = terminal_node(right_child)
    return node

  # 3] Check if number of observations in the left child at the current node is less than the minimum samples needed to make a split, which will be stored as min_samples_split. If so create a terminal node and return the tree
  # 4] Check if number of observations in the left child at the current node is less than the minimum samples needed to make a split, which will be stored as min_samples_split. If so create a terminal node and return the tree
  if len(left_child['y_bootstrap']) <= min_sample_split:
    node['left_split'] = node['right_split'] = terminal_node(left_child)
  else:
    node['left_split'] = find_split_point(left_child["X_bootstrap"], left_child["y_bootstrap"], max_features)
    split_node(node['left_split'], max_features, min_sample_split, max_depth, depth+1)

  # 5] Repeat step 3 and 4 for right child
  if len(right_child['y_bootstrap']) <= min_sample_split:
    node['right_split'] = node['left_split'] = terminal_node(right_child)
  else:
    node['right_split'] = find_split_point(right_child['X_bootstrap'], right_child['y_bootstrap'], max_features)
    split_node(node['right_split'], max_features, min_sample_split, max_depth, depth+1)


## **Parameters**
There are others parameters to consider when building a random forest, but these 4 will be the only ones we'll focus on.

1] n_estimators: (int) The number of trees in the forest.

2] max_features: (int) The number of features to consider when looking for the best split (typically (√p)

3] max_depth: (int) The maximum depth of the tree

4] min_samples_split: (int) The minimum number of samples required to split an internal node

## **Model Building**
 We first call **find_split_point** to create the very first split in our tree which we'll call *root_node*. Once we have a root node we can feed it into **split_node** which will recusively split each internal node until each branch has a terminal node.

Now that we can build a single tree we can finally build our random forest which will just be a collection of these trees.

When we call **random_forest** we'll need to specify n_estimators, max_features, max_depth, min_samples_split.

In [ ]:
def build_tree(X_bootstrap, y_bootstrap, max_depth, min_sample_split, max_features):
  root_node = find_split_point(X_bootstrap, y_bootstrap, max_features)
  split_node(root_node, max_features, min_sample_split, max_depth, 1)
  return root_node

def random_forest(X_train, y_train, n_estimators, max_features, max_depth, min_sample_split):
  tree_list = []  # all the trees dictionaries will be stored here
  oob_list = []   # oob error scores will be stored here

  for i in range (n_estimators):
    X_bootstrap, y_bootstrap, X_oob, y_oob = draw_bootstrap(X_train, y_train)
    y_bootstrap = np.array(y_bootstrap)
    y_oob = np.array(y_oob)


    tree = build_tree(X_bootstrap, y_bootstrap, max_depth, min_sample_split, max_features)
    tree_list.append(tree)

    oob_error = oob_score(tree, X_oob, y_oob)
    oob_list.append(oob_error)

  print(f"OOB estimate: {np.mean(oob_list)}")
  return tree_list



Given a input vector xi we can predict its class given a single tree with **predict_tree**. As a single tree consists of nested dictionaries which each represent a node we can let our xi flow through a tree by constantly checking if the split we're at contains another dictionary (node). Once we reach a left_split or right_split that does not contain any dictionary we've reached the terminal node and we can return the class.

In [ ]:
def predict_tree(tree, X_test):
  feature = tree['feature_index']

  if X_test[feature] <= tree['split_point']:
    if type(tree['left_split']) == dict:
      return predict_tree(tree['left_split'], X_test)
    else:
      value = tree['left_split']
      return value

  else:
    if type(tree['right_split']) == dict:
      return predict_tree(tree['right_split'], X_test)
    else:
      value = tree['right_split']
      return value

We'll repeat the above process for an input xi for all the trees in our ensemble and whichever class was returned more frequently will be the class predicted from our model.

In [ ]:
def predict(tree_list, X_test):
  pred_list = []

  for i in range (len(X_test)):
    for tree in tree_list:
      ensemble_preds = [predict_tree(tree, X_test.values[i])]
      final_pred = max(ensemble_preds, key = ensemble_preds.count)
    pred_list.append(final_pred)

  return np.array(pred_list)

Now that we have our model built we can fit it to our training data with **random_forest** and predict on our training data

In [ ]:
model = random_forest(X_train, y_train, n_estimators=100, max_features=3, max_depth=10, min_sample_split=2)

OOB estimate: 0.3336712305177697


In [ ]:
predictions = predict(model, X_test)
accuracy = sum(predictions == y_test) / len(y_test)
print(accuracy)

0.6201117318435754
